In [1]:
import requests
# re - модуль для работы с регулярными выражениями
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawl(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
            
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        # re.sub() для замены всех ' +' на ' '
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 1000:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"
        # re.findall() - найти все вхождения
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawl(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawl([start])

In [4]:
index = {i:k for i, k in enumerate(texts)}

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [12]:
path = 'C:\\Users\\User\\Downloads\\InfoSearch\\InfoSearch_1\\index.txt'  # путь указывается индивидуально
open(path).read()

'0: https://m.habr.com/ru/post/543540/comments/\n1: https://m.habr.com/ru/company/itsoft/blog/543554/comments/\n2: https://m.habr.com/ru/post/543532/\n3: https://m.habr.com/ru/post/543550/\n4: https://m.habr.com/ru/post/543570/\n5: https://m.habr.com/ru/post/543546/comments/\n6: https://m.habr.com/ru/post/543534/\n7: https://m.habr.com/ru/post/543540/\n8: https://m.habr.com/ru/post/543586/\n9: https://m.habr.com/ru/post/543536/\n10: https://account.habr.com/info/confidential/\n11: https://m.habr.com/ru/company/vasexperts/blog/543300/\n12: https://m.habr.com/ru/company/itsoft/blog/543554/\n13: https://m.habr.com/ru/post/543550/comments/\n14: https://m.habr.com/ru/post/543542/comments/\n15: https://m.habr.com/ru/post/543552/\n16: https://m.habr.com/ru/post/543496/\n17: https://m.habr.com/ru/post/543570/comments/\n18: https://m.habr.com/ru/post/543580/\n19: https://account.habr.com/info/agreement\n20: https://m.habr.com/ru/company/audiomania/blog/541106/\n21: https://m.habr.com/ru/post/54